In [74]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # Para a barra de progresso
from io import StringIO
import time

url_acoes = "https://investidor10.com.br/acoes/"

def buscar_fundamentalista_por_url(url):
    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    # Aguarda o carregamento inicial da página
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "img-logo")))
    time.sleep(3)

    def transformar_tabela(id):
        try:
            table = driver.find_element(By.ID, id)
            table_html = table.get_attribute('outerHTML')
            df = pd.read_html(StringIO(table_html))[0]
        except:
            df = pd.DataFrame()

        return df
    
    # Extrai as tabelas desejadas
    indicadores = transformar_tabela("table-indicators-history")
    resultados = transformar_tabela("table-balance-results")
    balanco = transformar_tabela("table-balance-sheet")

    driver.quit()

    return url, indicadores, resultados, balanco


def buscar_fundamentalista_em_paralelo(urls):
    resultados = []
    
    # Utiliza ThreadPoolExecutor para paralelismo
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(buscar_fundamentalista_por_url, url): url for url in urls}
        
        # Barra de progresso com tqdm
        for future in tqdm(as_completed(future_to_url), total=len(urls), desc="Scraping URLs"):
            try:
                resultados.append(future.result())
            except Exception as e:
                url_with_error = future_to_url[future]
                print(f"Erro ao processar {url_with_error}: {e}")
    
    return resultados

In [75]:
# Lista de URLs para scraping
df_ativos = pd.read_csv('./data/ativos/tickers.csv')
df_ativos['url'] = df_ativos['ticker'].apply(lambda x: f'{url_acoes}{x.lower()}')
urls = df_ativos['url'].tolist()[:5]

In [76]:
# Executa o scraping em paralelo
res = buscar_fundamentalista_em_paralelo(urls)

Scraping URLs: 100%|██████████| 5/5 [00:28<00:00,  5.71s/it]


In [77]:
df_indicadores = pd.DataFrame()
df_resultados = pd.DataFrame()
df_balanco = pd.DataFrame()

lista_ativos_problema = []

for url, indicadores, resultados, balanco in res:
    ativo = url.split('/')[-1].upper()
    print(f"Processando {ativo}...")
    try:
        ind = indicadores.rename(columns={'Unnamed: 0': 'Indicador'}).set_index('Indicador')
        ind['ticker'] = ativo
        df_indicadores = pd.concat([df_indicadores, ind])
    except KeyError:
        print(f"Erro ao processar indicadores de {ativo}")
        lista_ativos_problema.append(ativo)

    try:
        dre = resultados.rename(columns={'#': 'Descrição'})
        dre = dre[dre.columns[~dre.columns.str.contains('%')]]
        dre.set_index('Descrição', inplace=True)
        dre['ticker'] = ativo
        df_resultados = pd.concat([df_resultados, dre])
    except KeyError:
        print(f"Erro ao processar resultados de {ativo}")
        lista_ativos_problema.append(ativo)

    try:
        bal = balanco.rename(columns={'#': 'Descrição'})
        bal = bal[bal.columns[~bal.columns.str.contains('%')]]
        bal.set_index('Descrição', inplace=True)
        bal['ticker'] = ativo
        df_balanco = pd.concat([df_balanco, bal])
    except KeyError:
        print(f"Erro ao processar balanço de {ativo}")
        lista_ativos_problema.append(ativo)

lista_ativos_problema = list(set(lista_ativos_problema))

Processando WLMM3...
Processando VAMO3...
Processando AZEV4...
Processando VLID3...
Processando ETER3...


In [78]:
df_indicadores

,Atual,2023,2022,2021,2020,2019,ticker
Indicador,,,,,,,
P/L,705,710,667,1171,1473,950,WLMM3
P/RECEITA (PSR),030,034,040,067,081,044,WLMM3
P/VP,129,116,126,228,173,113,WLMM3
DIVIDEND YIELD (DY),"2,19%","3,88%","5,66%","1,68%","1,45%","1,46%",WLMM3
PAYOUT,"16,26%","29,07%","29,96%","20,73%","22,57%","14,63%",WLMM3
...,...,...,...,...,...,...,...
PATRIMÔNIO / ATIVOS,061,059,055,056,029,-001,ETER3
PASSIVOS / ATIVOS,039,041,045,044,071,101,ETER3
LIQUIDEZ CORRENTE,205,192,252,354,220,102,ETER3
